In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import haiku as hk

In [2]:
import metaaf

## 制作数据集

When you make a metaaf dataset, you write a vanilla pytorch dataset. The dataset should not use jax and must return a dictionary with the keys "signals" and "metadata". We enforce this format since the "signals" are automatically segmented and buffered. In this example, we make a simple system identification dataset which returns signals but not metadata.

这里通过Pytorch的`dataset`方法构造metaaf的数据集。

**注意**：数据集不能使用`jax`且必须返回一个包含`signals`和`metadata`的字典。为了实现字典数据返回格式，`signals`将会被自动分段和缓冲。

In [ ]:
from tkinter import N
from torch.utils.data import Dataset

# 构造pytorch数据集
class SystemIDDataset(Dataset):
    def __init__(self, total_size=1024, N=4096, sys_order=32):
        self.N = N
        self.sys_order = sys_order
        self.total_size = total_size

    def __len__(self):
        return self.total_size

    def __getitem__(self, idx):
        # 系统构造，其实不同的系统就是设计不同的滤波器系数矩阵w
        w = np.random.normal(size=self.sys_order) / self.sys_order

        # 系统输入信号u，即滤波器的输入
        u = np.random.normal(size=self.N)

        # 输出：系统输出信号d，即期望的目标响应(target or desired response)
        # 通过np.convole方法返回线性离散1D卷积序列
        d = np.convolve(w, u)[: self.N]

        # meta自适应滤波数据集返回一个包含如下内容的字典：
        # （1）"signals"：该信号会被自动添加到缓冲器
        # （2）"metadata"：由用户自行管理
        return {
            "signals": {
                "u": u[:, None],    # 时间X通道
                "d": d[:, None],
            },
            "metadata": {},
        }

## 制作滤波器

滤波器是用来适应输入信号的。Metaaf为各种常见的STFT处理管道提供了封装好的代码：`Overlap-Save`、`Overlap-Add`和`Weighted Overlap-Add`。另外，通过`buffering`框架，我们也可以构建自己的滤波器。

所有的滤波器都是在`haiku`库中编写的。当使用封装好的代码，滤波器必须返回一个包含`out`键的字典，它是后续STFT处理需要的数据。

In [ ]:
from metaaf.filter import OverlapSave

# 滤波器继承于overlap save modules
class SystemID(OverlapSave, hk.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # 选择分析窗：
        # 这里的self.window_size是OverlapSave中的类变量，由于这里构造的滤波器
        # 继承于OverlapSave,所以可以直接定义self.window_size
        self.analysis_window = jnp.ones(self.window_size)

    # 由于我们使用的是OLS基类，x和y是stft域的输入
    # 因此，The filter msut take the same inputs provided in its _fwd function.
    def __ols_call__(self, u, d, metadata):
        # Collect a buffer size anti-aliased filter
        # 使用get_filter方法构造滤波器
        w = self.get_filter(name="w")

        # 系统估计响应（extimated response）y和系统误差e
        # this is n_frames x n_freq x channels or 1 x F x 1 here
        y = w * u
        e = y - d

        return {
            "out": y[0],
            "u": u,
            "d": d,
            "e": e,
            "loss": jnp.vdot(e, e).real / (e.size), # The MSE of the prediction
        }

# 为此下面提供一个封装的包，以实现通过Haiku包将对象转换为函数。
# 封装的函数必须从数据集中获取相同命名作为输入
def _SystemID_fwd(u, d, metadata=None, int_data=None, **kwargs):
    gen_filter = SystemID(**kwargs)
    return gen_filter(u=u, d=d)

# 我们还需要定义滤波器的损失
def filter_loss(out, data_samples, metadata):
    return out["loss"]

# 同时我们还需要定义meta损失
def meta_loss(losses, outputs, data_samples, metadata, outer_leanable):
    EPS = 1e-9
    return jnp.log(jnp.mean(jnp.abs(outputs - data_samples["d"]) ** 2) + EPS)

## 初始化Meta训练

Next, we setup all filter keyword arguments. These will be used by the OLS baseclass to correctly buffer inputs and run the online STFT processing. These can also be done via argparse, since all metaaf modules have argparse utilities. Here, we use dictionaries for simplicity.

**设置所有滤波器的关键字参数：**

这些参数将被用于OLS基类输入的准确缓存以及在线STFT的处理。由于所有`MetaAF`模块都具有`argparse`工具包，因此，这些操作是通过`argparse`完成的。为了简单起见，这里使用字典形式的数据。

In [ ]:
from metaaf.data import NumpyLoader

# 所有滤波器的初始化参数，这些参数将被用于STFT的处理参数
filter_kwargs = {
    "n_frames": 1,
    "n_in_chan": 1,
    "n_out_chan": 1,
    "window_size": 64,
    "hop_size": 32,
    "pad_size": 0,
    "is_real": True,
}

# 优化器参数
optimizer_kwargs = {
    "h_size": 16,
    "n_layers": 1,
    "lam_1": 1e-2,
    "input_transform": "log1p",
}

# 使用封装好的NumpyLoader类包，初始化训练需要的dataloader数据
train_loader = NumpyLoader(SystemIDDataset(total_size=1024), batch_size=16)
val_loader = NumpyLoader(SystemIDDataset(total_size=1024), batch_size=32)
test_loader = NumpyLoader(SystemIDDataset(total_size=1024), batch_size=32)

Now, we create a metaaf system. This system manages the training and will later provide inference utilities. We need to pass it the forward functions, losses, keyword arguments as well as the dataloaders. We'll set some optimizer options. For more advanced functionality, we can write our own forward passes, overridde other options, and even pass in training callbacks. These could do things like save checkpoints, log outputs, and more.

现在，**创建一个`MetaAF`系统：**
